# Structured Output with Anthropic
In order to build reliable pipelines in which LLMs consistently return output in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will conform to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

**To generate structured outputs with Anthropic, we'll use the library `instructor`**

In [1]:
!pip install anthropic instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 1.9 MB/s eta 0:00:00


In [2]:
import anthropic
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List, Optional
import instructor

ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

We use pydantic to create data models.
- Here we would like to imagine cities for a role-playing-game scenarios.

In [3]:
class City(BaseModel):
    name: str = Field(..., description="Name of the city. Can be inspired from Oriental, Asian, Russian, Amercian, European styles")
    biotope: str = Field(..., description="The natural surroundings or environmental features of the city")
    economy: str = Field(..., description="The primary industries, economic status, or key exports/imports")
    culture: str = Field(..., description="Traditions, festivals, social norms, and attitudes towards outsiders")
    military: Optional[str] = Field(None, description="Details about the city's defense systems, military force, or recent conflicts")
    technology: Optional[str] = Field(None, description="Level of technological or magical advancement, and its accessibility")
    notable_features: Optional[List[str]] = Field(None, description="List of unique landmarks, key buildings, or hidden locations")
    population: Optional[str] = Field(None, description="Demographics, dominant species, and population size")
    mood: Optional[str] = Field(None, description="General atmosphere, sensory details, or challenges faced by residents")

We can inject some initial ideas we might have

In [4]:
idea_seeds = [
    'A city name Aquabah',
    'A city named Kniga where human built mechanical creatures and machine to assist them',
    'something with Italian/German vibes',
    'Very cold place',
    'Desert'
]

Note below how:
- we pass the `antropic_client` to **`instructor`**
- we loop through our initial ideas
- inject ideas into the user prompt template
- have high temperature for good creativity
- pass the **`City`** pydantic data schema to the `response_model` argument.

In [9]:
antropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
client = instructor.from_anthropic(
                antropic_client, mode=instructor.Mode.ANTHROPIC_TOOLS
            )

cities = []
for idea in idea_seeds:
    city = client.messages.create(
        model="claude-3-5-haiku-latest",
        max_tokens=2056,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful table-top RPG gamemaster assistant, \
                and a world class scenario writer."},
            {
                "role": "user",
                "content": f"Imagine a city on the basis of the following idea: {idea}. \
                 Be creative. \
                 Provide only one sentence per attribute of the city. Be very concise. \
                 Use language adequat for teenagers."
            },
        ],
        temperature=0.9,
        response_model=City,
    )
    cities.append(city)

Now we can see the 5 `cities` objects which have been created.

In [10]:
len(cities)

5

In [11]:
from pprint import pprint

In [12]:
print(cities[3].name, '\n')
pprint(cities[3].model_dump())

Frostholm 

{'biotope': 'A harsh, frozen landscape of eternal winter with jagged ice '
            'mountains and frozen tundra stretching as far as the eye can see',
 'culture': 'Residents are hardcore survivors who celebrate their annual Ice '
            'Trials - an extreme coming-of-age ritual where teens prove they '
            "can handle the city's insane cold",
 'economy': 'Survival depends on fur trading, ice mining, and advanced '
            'cold-resistant technology that lets people make bank in the most '
            'brutal winter conditions',
 'military': 'Elite frost warriors trained in guerrilla tactics who use the '
             'brutal environment as their ultimate weapon against any '
             'potential invaders',
 'mood': 'A constant vibe of gritty determination mixed with bone-chilling '
         'isolation that keeps everyone close-knit and fiercely protective',
 'name': 'Frostholm',
 'notable_features': ['Obsidian Ice Citadel',
                      'Fro

Now what? **We could use those generate cities and generate a couple of NPCs for each city** 😱
As you can see, structured output are powerful!

Learn more about instructor here: https://python.useinstructor.com/